In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[=================================================-] 99.2% 1649.2/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [3]:
model_path ='/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz'
from gensim.models.keyedvectors import KeyedVectors
w2v_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [4]:
import numpy as np
!pip install python-docx
!pip install transformers
!pip install --upgrade transformers
!pip install --upgrade tensorflow
!pip install sentencepiece

     |████████████████████████████████| 5.5MB 4.9MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=b3b145e5c10ba0686e5457907574e4e8049d5875b448e1e1e11a1b6dffc5a120
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx
     |████████████████████████████████| 1.5MB 5.6MB/s 
     |████████████████████████████████| 890kB 22.5MB/s 
     |████████████████████████████████| 2.9MB 26.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a0ec764c2e278b2443236005eb4a3f7378a4fb14e0f73648d164b52ea6e20573
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (4.1.1)
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages

In [6]:
class DocSim:
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []

    def vectorize(self, doc: str) -> np.ndarray:
        """
        Identify the vector values for each word in the given document
        :param doc:
        :return:
        """
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        vector = np.mean(word_vecs, axis=0)
        return vector

    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim

    def calculate_similarity(self, source_doc, target_docs=None, threshold=0):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append( sim_score)
            # Sort results by score in desc order
            #results.sort(key=lambda k: k["score"], reverse=True)

        return results

In [7]:
stopwords_path="drive/MyDrive/cohesion/stopwords.txt"
with open(stopwords_path, 'r') as fh:
    stopwords = fh.read().split(",")

In [8]:
ds = DocSim(w2v_model,stopwords=stopwords)

In [9]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

Some weights of the model checkpoint at t5-small were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
device = torch.device('cpu')

In [11]:
import re
import pandas as pd
from docx import Document

In [12]:
def countX(lst, x): 
    return lst.count(x) 

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [32]:
def gin(prob,paragraph):
  paar=paragraph.replace("\n\n","\n")
  
  para=paar.split("\n")
  
  while '' in para:para.remove('')
    #para.remove('')
    #para.remove(' ')
  #except Exception:
    #abc=len(para)
  abc=len(para)
  #print(para)
  #print(abc)
  #print(prob)
  lines=[]
  lines_res=[]
  for j in range(0,abc):
    z=para[j]
    y=z.split(".")
    ee=y[0]
  #xe=y[1].join(y[:])
    lines.append(ee)
    for e in y:
      s=1
      le=len(y)
      att=''.join(y[1:le])
    lines_res.append(att)
  #print(lines_res)

  summarys=[]
  for i in para:
      text=i
      preprocess_text = text.strip().replace("\n","")
      t5_prepared_Text = "summarize: "+preprocess_text
      #print ("original text preprocessed: \n", preprocess_text)

      tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


      # summmarize 
      summary_ids = model.generate(tokenized_text,
                                      num_beams=4,
                                      no_repeat_ngram_size=2,
                                      min_length=30,
                                      max_length=100,
                                      early_stopping=True)

      output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
      summarys.append(output)

  texts=prob
  preprocess_texts = texts.strip().replace("\n","")
  t5_prepared_Texts = "summarize: "+preprocess_texts
      #print ("original text preprocessed: \n", preprocess_text)

  tokenized_texts = tokenizer.encode(t5_prepared_Texts, return_tensors="pt").to(device)


      # summmarize 
  summary_idss = model.generate(tokenized_texts,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

  outputs = tokenizer.decode(summary_idss[0], skip_special_tokens=True)
  #print(outputs)
 #first line of para1
  texts_first=lines[0]
  preprocess_texts_first = texts_first.strip().replace("\n","")
  t5_prepared_Texts_first = "summarize: "+preprocess_texts_first
      #print ("original text preprocessed: \n", preprocess_text)

  tokenized_texts_first = tokenizer.encode(t5_prepared_Texts_first, return_tensors="pt").to(device)


      # summmarize 
  summary_idsss = model.generate(tokenized_texts_first,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

  outputs_firstline = tokenizer.decode(summary_idsss[0], skip_special_tokens=True)
  #print(outputs_firstline)
#remaining line of para1
  texts_rest=lines_res[0]
  preprocess_texts_rest = texts_rest.strip().replace("\n","")
  t5_prepared_Texts_rest = "summarize: "+preprocess_texts_rest
      #print ("original text preprocessed: \n", preprocess_text)

  tokenized_texts_rest = tokenizer.encode(t5_prepared_Texts_rest, return_tensors="pt").to(device)


      # summmarize 
  summary_idssss = model.generate(tokenized_texts_rest,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

  outputs_rest = tokenizer.decode(summary_idssss[0], skip_special_tokens=True)
  #print(lines_res[0])
  #print(outputs_rest)




  lines_sum=[]
  lines_res_sum=[]
  for c in range(0,abc):
    w=summarys[c]
    yy=w.split(".")
    eee=yy[0]
    #xe=y[1].join(y[:])
    lines_sum.append(eee)
    for es in yy:
      ss=1
      lee=len(yy)
      attt=''.join(yy[1:lee])
    lines_res_sum.append(attt)
  #print(summarys[0])
 # print(lines_sum)
  #print(lines_res_sum)
  #while '' in lines_sum:lines_sum.remove('')
  #while '' in lines_res_sum:lines_res_sum.remove('')

  sim_scores1=ds.calculate_similarity(outputs, outputs_firstline)
  sim_scores2=ds.calculate_similarity(outputs, outputs_rest)
  try:
    sim_scores3=ds.calculate_similarity(outputs, summarys[1])
  except Exception as e:
    sim_scores3=0
  try:
    sim_scores4=ds.calculate_similarity(outputs, summarys[2])
  except Exception as e:
    sim_scores4=0
  try:
    sim_scores5=ds.calculate_similarity(summarys[1], summarys[2])
  except Exception as e:
    sim_scores5=0
##linking words

  document1 = Document('drive/MyDrive/cohesion/link.docx')
  link=[]
  hello=['','\xa0']
  for paras in document1.paragraphs:
      if paras.text not in hello:
      #if paras.text != '':
        link.append(paras.text)


  length=len(para)
  words=[]
  parass=[]
  for ss in range(0,length):
      essay=para[ss]
      for j in link:
          if re.search(j,essay,re.IGNORECASE):
              words.append(j)
              parass.append(ss)
    
    #valu=Counter(parass)

  coo=[]
  for co in range(0,length):
    coo.append(countX(parass,co))
  a_list=zip(words,parass)
  final_list=list(a_list)
  df=pd.DataFrame()
  df['Words']=words
  df['Para_number']=parass
  tot_num=df.Para_number.count()

  #results1=[]
  #results2=[]
  try:
    results1=[]
    for me in coo:
      #res_avg=(me/tot_num)
      res1=(me/tot_num)**2
      results1.append(res1)
      #results2.append(res_avg)
    total=sum(results1)
    total_avg= tot_num/(length)
  except Exception as e:
    total=0
    total_avg=0


##reference words
  document2 = Document('drive/MyDrive/cohesion/reference.docx')
  reference=[]
  hello=['','\xa0']
  for paras in document2.paragraphs:
      if paras.text not in hello:
      #if paras.text != '':
        reference.append(paras.text)

  length=len(para)
  wordsr=[]
  parassr=[]
  for ssr in range(0,length):
      essayr=para[ssr]
      for jr in reference:
          if re.search(jr,essayr,re.IGNORECASE):
              wordsr.append(jr)
              parassr.append(ssr)
    
    #valu=Counter(parass)
  coor=[]
  for cor in range(0,length):
    coor.append(countX(parassr,cor))
  a_listr=zip(wordsr,parassr)
  final_listr=list(a_listr)
  dfr=pd.DataFrame()
  dfr['Words']=wordsr
  dfr['Para_number']=parassr
  tot_numr=dfr.Para_number.count()

  try:
    results2=[]
    for mer in coor:
      res2=(mer/tot_numr)**2
      results2.append(res2)
    totalr=sum(results2)
    avg_ref= tot_numr/(length)
    countr=dfr.groupby(['Para_number']).size().reset_index(name='counts')
  except Exception as e:
    totalr=0
    countr=0

#conclusion 
  document3 = Document('drive/MyDrive/cohesion/conclusion.docx')
  concl=[]
  hello=['','\xa0']
  for paras in document3.paragraphs:
      if paras.text not in hello:
      #if paras.text != '':
        concl.append(paras.text)
  try:
    sim_scores6 = ds.calculate_similarity(summarys[abc-1],outputs_rest)
  except Exception as e:
    sim_scores6=0
  #length=len(para)
  wordsc=[]
  parassc=[]
  essayc=para[-1]
  #no=essayc.split()
  #jj=len(no)
  try:
    for jc in concl:
        if re.search(jc,essayc,re.IGNORECASE):
                wordsc.append(jc)
                  #parassc.append(ssc)
    #print(wordsc)
    no_concl=len(wordsc)
  except Exception as e:
    no_concl=0

  #print(para[0])
  #print(lines[0])
  #print(lines_res[0])
  #print(outputs_firstline)
  #print(outputs_rest)
  #return exe
  results=[abc,str(sim_scores1),str(sim_scores2),str(sim_scores3),str(sim_scores4),str(sim_scores5),str(no_concl),str(sim_scores6),str(total_avg),str(total),str(avg_ref),str(totalr)]
  #results=[abc,a1,b1,c1,d1,e1,str(no_concl),f1,str(total_avg),str(total),str(avg_ref),str(totalr)]
  return results


  #print(para[0])
  #print(lines[0])
  #print(lines_res[0])
 # print(outputs_firstline)
  #print(outputs_rest)
  #print(lines_sum)
  #print(lines_res_sum)
  #print(sim_scores1)
  #print(sim_scores2)
  #print(sim_scores3)
  #print(sim_scores4)
  #print(sim_scores5)
  #print(jj)

  #print(outputs)

#be=prob,ae=para
#rr=gin(be,ae)  

In [33]:
df=pd.read_excel("drive/MyDrive/cohesion/CC_ANN_Model.xlsx")
df
#len(df)



,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,Code,Question,Essay,Overall Score,CC,Lex,Grammar,Task Achv,No of Paragraph,"Similarity Score (Problem Statement, 1st Sente...","Similarity Score (Problem Statement, Remaining...","Similarity Score (Problem Statement, Para-2)","Similarity Score (Problem Statement, Para-3)","Similarity Score (Para-2, Para-3)",Presence of Concluding words/phrases,"Similarity Score (Last Para, Remaining part of...",Avg Linking word per para,HHE Score Linking Words,Avg Reference word per para,HHE Score Reference Words
1,IC1,Some people believe that technology has made m...,Experts throughout both the developing and dev...,7,7,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IC2,People are using a lot of online language tran...,The importance and popularity of web-based lan...,6.5,7,7,7,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IC3,These days some people spend a lot of money on...,The issue of whether to spend one’s hard earne...,7,7,7,7,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IC4,Online education and training is becoming incr...,Enthusiasm for digital technology is at an all...,6.5,6,7,7,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,AA196,Modern societies need specialists in certain f...,They tell me; Love never dies!\nI ask them; Do...,7,6.5,6.5,6.5,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292,AA197,How do you think nuclear technologies will evo...,"from my prespective, i think that the utilizat...",6.5,6.5,6.5,7,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293,AA198,what is those good means something really not ...,One year access to the entire PrepScholar prog...,5,5,5,5.5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,AA199,A foreign visitor has only one day to spend in...,If I want to talk about my country that will t...,6,6.5,6,6,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
#


In [ ]:
ra=range(1,51)
fin=[]
for i in ra:
  prob=df['Unnamed: 1'][i]
  paras=df['Unnamed: 2'][i]
  resultss=gin(prob,paras)
  fin.append(resultss)
#print(fin)

  

In [31]:
#print(str(df['Unnamed: 2'][71]))

It is undoubtedly that the world encounters the new challenge of enlarged need for nourishment in comparison with the last century. This state might have two grounds - the huge increase of population all over the world and notable climate changes such as global warming. Despite the current status of this concern it might be rectified by several â€œrescueâ€ programs in the manner of a proper field cultivation or using state-of-the-art technologies in agricultural and biological sciences.
	Food demand is meant to increase anywhere up to 98 percent by 2050. This may become a cause for changing markets that no one has seen before. In fact, farmers and businesses did not manage to catch it up, so they faced the low offerings at the time of tremendous demand. Unfortunately, the climate changes have also contributed negatively, so that it has led to a reduction in possible food production pace. There are a variety of forms of them from global warming caused by high CO2 emissions up to a depl

In [30]:
print(fin)

[[4, '[0.54174495]', '[0.56553113]', '[0.72577715]', '[0.51141906]', '[0.5595054]', '1', '[0.68777305]', '3.25', '0.2781065088757397', '4.5', '0.2777777777777778']]


In [62]:
par_s=[]
sim_sco1=[]
sim_sco2=[]
sim_sco3=[]
sim_sco4=[]
sim_sco5=[]
concl_wo=[]
sim_sco6=[]
avg_lin=[]
hhe_link=[]
avg_ref=[]
hhe_ref=[]

for i in fin:
  par_s.append(i[0])
  sim_sco1.append(i[1])
  sim_sco2.append(i[2])
  sim_sco3.append(i[3])
  sim_sco4.append(i[4])
  sim_sco5.append(i[5])
  concl_wo.append(i[6])
  sim_sco6.append(i[7])
  avg_lin.append(i[8])
  hhe_link.append(i[9])
  avg_ref.append(i[10])
  hhe_ref.append(i[11])

print(sim_sco2)

['[0.578091]', '[0.7225043]', '[0.67608]', '[0.54487336]', '[0.25872934]', '[0.6841996]', '[0.3885041]', '[0.5217158]', '[0.40795892]', '[0.12117803]', '[0.6023805]', '[0.5624409]', '[0.6701355]', '[0.28957674]', '[0.4614207]', '[0.37801525]', '[0.23402713]', '[]', '[0.6727857]', '[0.5278691]', '[]', '[0.39966443]', '[0.59919316]', '[0.28310516]', '[0.3654935]', '[0.47415784]', '[0.8546635]', '[0.2584248]', '[0.8541606]', '[]', '[0.59675163]', '[]', '[0.8404949]', '[0.52642524]', '[0.46929082]', '[0.42316654]', '[]', '[0.95697767]', '[0.4017174]', '[0.44837576]', '[0.112542726]', '[0.5388785]', '[0.27483836]', '[0.46498215]', '[0.18961617]']


In [64]:
df_new=pd.DataFrame()
df_new['Code']=df['Unnamed: 0'][251:296]
df_new['Question']=df['Unnamed: 1'][251:296]
df_new['Essay']=df['Unnamed: 2'][251:296]
df_new['Overall score']=df['Unnamed: 3'][251:296]
df_new['CC']=df['Unnamed: 4'][251:296]
df_new['Lex']=df['Unnamed: 5'][251:296]
df_new['Grammer']=df['Unnamed: 6'][251:296]
df_new['task achv']=df['Unnamed: 7'][251:296]
df_new['No of paragraph']=par_s
df_new['sim score(problem statement & 1st sentence']=sim_sco1
df_new['Similarity Score (Problem Statement, Remaining part of Para-1']=sim_sco2
df_new['Similarity Score (Problem Statement, Para-2)']=sim_sco3
df_new['Similarity Score (Problem Statement, Para-3)']=sim_sco4
df_new['Similarity Score (Para-2, Para-3)']=sim_sco5
df_new['Presence of Concluding words/phrases']=concl_wo
df_new['Similarity Score (Last Para, Remaining part of Para-1)']=sim_sco6
df_new['Avg Linking word per para']=avg_lin
df_new['HHE Score Linking Words']=hhe_link
df_new['Avg Reference word per para']=avg_ref
df_new['HHE Score Reference Words']=hhe_ref

In [65]:
df_new
df_new.to_excel("drive/MyDrive/cohesion/update/final/output_final_1-51.xlsx")

TypeError: ignored

[[1, '[0.4385118]', '[0.44848222]', '0', '0', '0', '3', '0', '13.0', '1.0', '7.0', '1.0']]


"\n\nPeople learn and develop throughout their entire lives. I think that in our modern world it is very essential to be familiar with computer technology. So, if I had a chance to give a child a gift it would be a computer. I think that computers play an essential role in our lives and they bring many benefits to our society. Moreover, children can learn by use of computers. In the following paragraphs I will give my reasons to support my answer.\n\nFirst of all, by use of computers children can play many games, which help to improve children's ability to think logically, think about their next step, etc. Moreover, playing games develop many important qualities such as attention, patience, persistent, etc. Second of all, computers help children to learn more about anything by use of Internet. They can find new friends even from another country. Children will improve their communication skills, gain more knowledge and experience. Also, children have a great opportunity to learn more ab

In [21]:
def gini(prob,paragraph):
  para=paragraph.split(".\n")
  abc=len(para)
  print(para,"\n")
  print(abc)
  #print(prob)
  lines=[]
  lines_res=[]
  for j in range(0,abc):
    z=para[j]
    y=z.split(".")
    ee=y[0]
  #xe=y[1].join(y[:])
    lines.append(ee)
    for e in y:
      s=1
      le=len(y)
      att=''.join(y[1:le])
    lines_res.append(att)
  #print(lines_res)

  summarys=[]
  for i in para:
      text=i
      preprocess_text = text.strip().replace("\n","")
      t5_prepared_Text = "summarize: "+preprocess_text
      #print ("original text preprocessed: \n", preprocess_text)

      tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


      # summmarize 
      summary_ids = model.generate(tokenized_text,
                                      num_beams=4,
                                      no_repeat_ngram_size=2,
                                      min_length=30,
                                      max_length=100,
                                      early_stopping=True)

      output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
      summarys.append(output)

  texts=prob
  preprocess_texts = texts.strip().replace(".","")
  t5_prepared_Texts = "summarize: "+preprocess_texts
      #print ("original text preprocessed: \n", preprocess_text)

  tokenized_texts = tokenizer.encode(t5_prepared_Texts, return_tensors="pt").to(device)


      # summmarize 
  summary_idss = model.generate(tokenized_texts,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

  outputs = tokenizer.decode(summary_idss[0], skip_special_tokens=True)
  print(output)
  lines_sum=[]
  lines_res_sum=[]
  for c in range(0,abc):
    w=summarys[c]
    yy=w.split(".")
    eee=yy[0]
    #xe=y[1].join(y[:])
    lines_sum.append(eee)
    for es in yy:
      ss=1
      lee=len(yy)
      attt=''.join(yy[1:lee])
    lines_res_sum.append(attt)

  sim_scores1=ds.calculate_similarity(outputs, lines_sum[0])
  sim_scores2=ds.calculate_similarity(outputs, lines_res_sum[0])
  try:
    sim_scores3=ds.calculate_similarity(outputs, summarys[1])
  except Exception as e:
    sim_scores3=0
  try:
    sim_scores4=ds.calculate_similarity(outputs, summarys[2])
  except Exception as e:
    sim_scores4=0
  try:
    sim_scores5=ds.calculate_similarity(summarys[1], summarys[2])
  except Exception as e:
    sim_scores5=0
##linking words

  document1 = Document('drive/MyDrive/cohesion/link.docx')
  link=[]
  hello=['','\xa0']
  for paras in document1.paragraphs:
      if paras.text not in hello:
      #if paras.text != '':
        link.append(paras.text)


  length=len(para)
  words=[]
  parass=[]
  for ss in range(0,length):
      essay=para[ss]
      for j in link:
          if re.search(j,essay,re.IGNORECASE):
              words.append(j)
              parass.append(ss)
    
    #valu=Counter(parass)

  coo=[]
  for co in range(0,length):
    coo.append(countX(parass,co))
  a_list=zip(words,parass)
  final_list=list(a_list)
  df=pd.DataFrame()
  df['Words']=words
  df['Para_number']=parass
  tot_num=df.Para_number.count()

  #results1=[]
  #results2=[]
  try:
    results1=[]
    for me in coo:
      #res_avg=(me/tot_num)
      res1=(me/tot_num)**2
      results1.append(res1)
      #results2.append(res_avg)
    total=sum(results1)
    total_avg= tot_num/(length)
  except Exception as e:
    total=0
    total_avg=0


##reference words
  document2 = Document('drive/MyDrive/cohesion/reference.docx')
  reference=[]
  hello=['','\xa0']
  for paras in document2.paragraphs:
      if paras.text not in hello:
      #if paras.text != '':
        reference.append(paras.text)

  length=len(para)
  wordsr=[]
  parassr=[]
  for ssr in range(0,length):
      essayr=para[ssr]
      for jr in reference:
          if re.search(jr,essayr,re.IGNORECASE):
              wordsr.append(jr)
              parassr.append(ssr)
    
    #valu=Counter(parass)
  coor=[]
  for cor in range(0,length):
    coor.append(countX(parassr,cor))
  a_listr=zip(wordsr,parassr)
  final_listr=list(a_listr)
  dfr=pd.DataFrame()
  dfr['Words']=wordsr
  dfr['Para_number']=parassr
  tot_numr=dfr.Para_number.count()

  try:
    results2=[]
    for mer in coor:
      res2=(mer/tot_numr)**2
      results2.append(res2)
    totalr=sum(results2)
    avg_ref= tot_numr/(length)
    countr=dfr.groupby(['Para_number']).size().reset_index(name='counts')
  except Exception as e:
    totalr=0
    countr=0

#conclusion 
  document3 = Document('drive/MyDrive/cohesion/conclusion.docx')
  concl=[]
  hello=['','\xa0']
  for paras in document3.paragraphs:
      if paras.text not in hello:
      #if paras.text != '':
        concl.append(paras.text)
  try:
    sim_scores6 = ds.calculate_similarity(summarys[3],lines_res_sum[0])
  except Exception as e:
    sim_scores6=0
  #length=len(para)
  wordsc=[]
  parassc=[]
  essayc=para[-1]
  #no=essayc.split()
  #jj=len(no)
  try:
    for jc in concl:
        if re.search(jc,essayc,re.IGNORECASE):
                wordsc.append(jc)
                  #parassc.append(ssc)
    #print(wordsc)
    no_concl=len(wordsc)
  except Exception as e:
    no_concl=0

  
  #return exe
  results=[abc,str(sim_scores1),str(sim_scores2),str(sim_scores3),str(sim_scores4),str(sim_scores5),str(no_concl),str(sim_scores6),str(total_avg),str(total),str(avg_ref),str(totalr)]
  #results=[abc,a1,b1,c1,d1,e1,str(no_concl),f1,str(total_avg),str(total),str(avg_ref),str(totalr)]
  return results


  #print(summarys)
  #print(lines_sum)
  #print(lines_res_sum)
  #print(sim_scores1)
  #print(sim_scores2)
  #print(sim_scores3)
  #print(sim_scores4)
  #print(sim_scores5)
  #print(jj)

  #print(outputs)

#be=prob,ae=para
#rr=gin(be,ae)  

In [22]:
ra=range(71,72)
fin=[]
for i in ra:
  prob=df['Unnamed: 1'][i]
  paras=df['Unnamed: 2'][i]
  resultss=gini(prob,paras)
  fin.append(resultss)
#print(fin)

['It is undoubtedly that the world encounters the new challenge of enlarged need for nourishment in comparison with the last century. This state might have two grounds - the huge increase of population all over the world and notable climate changes such as global warming. Despite the current status of this concern it might be rectified by several â€œrescueâ€\x9d programs in the manner of a proper field cultivation or using state-of-the-art technologies in agricultural and biological sciences', '\tFood demand is meant to increase anywhere up to 98 percent by 2050. This may become a cause for changing markets that no one has seen before. In fact, farmers and businesses did not manage to catch it up, so they faced the low offerings at the time of tremendous demand. Unfortunately, the climate changes have also contributed negatively, so that it has led to a reduction in possible food production pace. There are a variety of forms of them from global warming caused by high CO2 emissions up t

In [210]:
ra=range(1,2)
fin=[]
for i in ra:
  prob=df['Unnamed: 1'][i]
  paras=df['Unnamed: 2'][i]
  resultss=gini(prob,paras)
  fin.append(resultss)
#print(fin)

['Experts throughout both the developing and developed world have debated whether the advent of sophisticated modern technology such as mobile phones, laptops and iPad have helped to enhance and improve people’s social lives or whether the opposite has become the case. Personally, I strongly advocate the former view. This essay will discuss both sides using examples from the UK government and Oxford University to demonstrate points and prove arguments', 'On the one hand there is ample, powerful, almost daily evidence that such technology can be detrimental especially to the younger generation who are more easily affected by it’s addictive nature and which can result in people feeling more isolated from the society. The central reason behind this is twofold, firstly, the invention of online social media sites and apps, such as Twitter and Facebook have reduced crucial face-to-face interactions dramatically. Through use of these appealing and attractive mediums, people feel in touch and 

In [221]:
ii=(df['Unnamed: 2'][202])
if "\n" in ii:
  print(str(ii))

Reaching the true information has been our concern since the beginning of human life. Since then, many scientists have investigated many concepts in different fields of life.  The collection of this knowledge has brought a dilemma that gathers scientists and business people to think about in which degree to mention information in their research or business-related articles. I personally think that it might be crucial to share a lot of information in business world while it is redundant to do it in an academic matter. The current essay will discuss this statement. 
To begin with, the main feature of being a leader in a business market is to be international as well as being interactive. So that,  companies can be aware of new ideas, trends and strategies to develop the quality and quantity of their work. That is why they need to process a lot of information to create new designs and maybe become unique in their market. 
On the other hand, the academic world needs specific information to